In [1]:
import re
import Queue
import json
import collections
from geopy.distance import vincenty
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint
import json
from SPARQLWrapper import SPARQLWrapper,TURTLE,JSON
import time
import csv

## find links to poi

In [7]:
DEPTH = 2
banned_properties = [u'http://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'http://dbpedia.org/ontology/wikiPageRedirects', \
                     u'http://www.w3.org/2002/07/owl#sameAs', \
                     u'http://xmlns.com/foaf/spec/', \
                     u'https://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'https://dbpedia.org/ontology/wikiPageRedirects', \
                     u'https://www.w3.org/2002/07/owl#sameAs', \
                     u'https://xmlns.com/foaf/spec/']

# create a queue for each run, this keeps things organized
resources_queues = [[] for i in range(DEPTH+1)]

# this set contains all resources downloaded
resources_known = set()
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

with open('source_data/dbpedia_poi_match.json', 'r') as pois:
    places = json.load(pois)
    
    distinct_dbpedia_uri = set([x[3] for x in places])
    print(len(distinct_dbpedia_uri))
    
    #insert all starting resources into the queue and the known set
    for uri in distinct_dbpedia_uri:
        resources_queues[0].append(uri)
        resources_known.add(uri)
    
    start = time.time()
    for run in range(DEPTH):
        #print(len(resources_queues[run]))
        count = 0
        for resource in resources_queues[run]:
            count += 1
            print(count)
            sparql.setQuery("""describe <%s>""" % resource)
            results = sparql.query().convert()
            for triple in results['results']['bindings']:
                if triple['p']['value'] not in banned_properties:
                    if triple['o']['value'] == resource and type(triple['s']['value']) == unicode \
                        and triple['s']['value'].startswith("http://dbpedia.org/resource/"):
                        
                        #print(triple['s']['value'])
                        if triple['s']['value'] not in resources_known:
                            resources_known.add(triple['s']['value'])
                            resources_queues[run + 1].append(triple['s']['value'])
                    elif triple['s']['value'] == resource and type(triple['o']['value']) == unicode \
                        and triple['o']['value'].startswith("http://dbpedia.org/resource/"):    
                        #print(triple['o']['value'])
                        if triple['o']['value'] not in resources_known:
                            resources_known.add(triple['o']['value'])
                            resources_queues[run + 1].append(triple['o']['value'])
    end = time.time()
    print('END')
    print(end - start)
    print(len(resources_known))

47
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
2

In [31]:
with open('tmp/depth_2.csv','wb') as out:
    csv_out=csv.writer(out,)
    for row in resources_known:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8])

## find links to artists

In [ ]:
DEPTH = 2
banned_properties = [u'http://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'http://dbpedia.org/ontology/wikiPageRedirects', \
                     u'http://www.w3.org/2002/07/owl#sameAs', \
                     u'http://xmlns.com/foaf/spec/', \
                     u'https://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'https://dbpedia.org/ontology/wikiPageRedirects', \
                     u'https://www.w3.org/2002/07/owl#sameAs', \
                     u'https://xmlns.com/foaf/spec/']

# create a queue for each run, this keeps things organized
resources_queues = [[] for i in range(DEPTH+1)]

# this set contains all resources downloaded
resources_known = set()
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)


with open('source_data/1_7_doremus_dbpedia_artists.csv','r') as artists:
    reader=csv.reader(artists,)
    next(reader, None)  # skip the headers
    
    distinct_dbpedia_uri = set([unicode(x[1],'utf-8') for x in reader])
    print(len(distinct_dbpedia_uri))
    
    #insert all starting resources into the queue and the known set
    for uri in distinct_dbpedia_uri:
        resources_queues[0].append(uri)
        resources_known.add(uri)
    
    start = time.time()
    for run in range(DEPTH):
        print(len(resources_queues[run]))
        count = 0
        for resource in resources_queues[run]:
            count += 1
            print(count)
            sparql.setQuery("""describe <%s>""" % resource)
            results = sparql.query().convert()
            for triple in results['results']['bindings']:
                if triple['p']['value'] not in banned_properties:
                    if triple['o']['value'] == resource and type(triple['s']['value']) == unicode and triple['s']['value'].startswith("http://dbpedia.org/resource/"):
                        #print(triple['s']['value'])
                        if triple['s']['value'] not in resources_known:
                            resources_known.add(triple['s']['value'])
                            resources_queues[run + 1].append(triple['s']['value'])
                    elif triple['s']['value'] == resource and type(triple['o']['value']) == unicode and triple['o']['value'].startswith("http://dbpedia.org/resource/"):
                        #print(triple['o']['value'])
                        if triple['o']['value'] not in resources_known:
                            resources_known.add(triple['o']['value'])
                            resources_queues[run + 1].append(triple['o']['value'])
    end = time.time()
    print('END')
    print(end - start)
    print(len(resources_known))

In [ ]:
with open('tmp/depth_2_artists.csv','wb') as out:
    csv_out=csv.writer(out,)
    for row in resources_known:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8])

## find links to works

In [ ]:
DEPTH = 2
banned_properties = [u'http://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'http://dbpedia.org/ontology/wikiPageRedirects', \
                     u'http://www.w3.org/2002/07/owl#sameAs', \
                     u'http://xmlns.com/foaf/spec/', \
                     u'https://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'https://dbpedia.org/ontology/wikiPageRedirects', \
                     u'https://www.w3.org/2002/07/owl#sameAs', \
                     u'https://xmlns.com/foaf/spec/']

# create a queue for each run, this keeps things organized
resources_queues = [[] for i in range(DEPTH+1)]

# this set contains all resources downloaded
resources_known = set()
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

unicode(row[1],'utf-8')


with open('source_data/1_6_doremus_dbpedia_works.csv','r') as artists:
    reader=csv.reader(artists,)
    next(reader, None)  # skip the headers
    
    distinct_dbpedia_uri = set([unicode(x[1],'utf-8') for x in reader])
    print(len(distinct_dbpedia_uri))
    
    #insert all starting resources into the queue and the known set
    for uri in distinct_dbpedia_uri:
        resources_queues[0].append(uri)
        resources_known.add(uri)
    
    start = time.time()
    for run in range(DEPTH):
        print(len(resources_queues[run]))
        count = 0
        for resource in resources_queues[run]:
            count += 1
            print(count)
            sparql.setQuery("""describe <%s>""" % resource)
            results = sparql.query().convert()
            for triple in results['results']['bindings']:
                if triple['p']['value'] not in banned_properties:
                    if triple['o']['value'] == resource and type(triple['s']['value']) == unicode and triple['s']['value'].startswith("http://dbpedia.org/resource/"):
                        #print(triple['s']['value'])
                        if triple['s']['value'] not in resources_known:
                            resources_known.add(triple['s']['value'])
                            resources_queues[run + 1].append(triple['s']['value'])
                    elif triple['s']['value'] == resource and type(triple['o']['value']) == unicode and triple['o']['value'].startswith("http://dbpedia.org/resource/"):
                        #print(triple['o']['value'])
                        if triple['o']['value'] not in resources_known:
                            resources_known.add(triple['o']['value'])
                            resources_queues[run + 1].append(triple['o']['value'])
    end = time.time()
    print('END')
    print(end - start)
    print(len(resources_known))

In [ ]:
with open('tmp/depth_2_works.csv','wb') as out:
    csv_out=csv.writer(out,)
    for row in resources_known:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8])

## check poi linked to nice


In [3]:
import re
import Queue
import json
import collections
from geopy.distance import vincenty
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint
import json
from SPARQLWrapper import SPARQLWrapper,TURTLE,JSON
import time
import csv

In [6]:
# this set contains all resources downloaded
resources_linked_nice = set()
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

with open('source_data/dbpedia_poi_match.json', 'r') as pois:
    places = json.load(pois)
    
    distinct_dbpedia_uri = set([x[3] for x in places])
    print(len(distinct_dbpedia_uri))
    
    #insert all starting resources into the queue and the known set
    
    for resource in distinct_dbpedia_uri:
        sparql.setQuery("""describe <%s>""" % resource)
        results = sparql.query().convert()
        for triple in results['results']['bindings']:
            if triple['o']['value'] == resource and type(triple['s']['value']) == unicode \
                and triple['s']['value'] == u"http://dbpedia.org/resource/Nice":
                    resources_linked_nice.add(resource)

            elif triple['s']['value'] == resource and type(triple['o']['value']) == unicode \
                and triple['o']['value'] == u"http://dbpedia.org/resource/Nice":
                    resources_linked_nice.add(resource)
                    
    print(len(resources_linked_nice))
    


47
12


In [8]:
difference = distinct_dbpedia_uri - resources_linked_nice
for res in difference:
    print(res)

http://dbpedia.org/resource/Allianz_Riviera
http://dbpedia.org/resource/Avenue_Jean_Médecin
http://dbpedia.org/resource/Conservatory_of_Nice
http://dbpedia.org/resource/Nice_Observatory
http://dbpedia.org/resource/Musée_d'art_moderne_et_d'art_contemporain
http://dbpedia.org/resource/Castle_of_Nice
http://dbpedia.org/resource/Drap
http://dbpedia.org/resource/Promenade_des_Anglais
http://dbpedia.org/resource/Observatoire_Oceanologique_de_Villefranche
http://dbpedia.org/resource/Palais_Nikaia
http://dbpedia.org/resource/Cimetière_du_Château
http://dbpedia.org/resource/Russian_Orthodox_Cemetery,_Nice
http://dbpedia.org/resource/Nice_Cathedral
http://dbpedia.org/resource/Hotel_Negresco
http://dbpedia.org/resource/Russian_Orthodox_Cathedral,_Nice
http://dbpedia.org/resource/Terra_Amata_(archaeological_site)
http://dbpedia.org/resource/Villa_Leopolda
http://dbpedia.org/resource/Jardin_botanique_de_la_Ville_de_Nice
http://dbpedia.org/resource/Sainte_Jeanne_d'Arc,_Nice
http://dbpedia.org/resour

In [ ]:
import rdflib
from rdflib import URIRef, Graph, Namespace
from rdflib.plugins.parsers.notation3 import N3Parser

g = Graph()
result = g.parse(file=open("lemon_example_fear.txt", "r"), format="application/n3")
print (g.serialize(format='xml'))

# dbpedia poi without coordinates

In [9]:
import re
import Queue
import json
import collections
from geopy.distance import vincenty
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint
import json
from SPARQLWrapper import SPARQLWrapper,TURTLE,JSON
import time
import csv

In [41]:
# this set contains all resources downloaded
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

distinct_dbpedia_category = set()
distinct_dbpedia_places = set()
queue = collections.deque()

distinct_dbpedia_category.add(u'http://dbpedia.org/resource/Category:Nice')
queue.append(u'http://dbpedia.org/resource/Category:Nice')
while len(queue) > 0:
    resource = queue.popleft()
    sparql.setQuery("""describe <%s>""" % resource)
    results = sparql.query().convert()
    for triple in results['results']['bindings']:
        if triple['o']['value'] == resource \
        and triple['p']['value'] == u"http://www.w3.org/2004/02/skos/core#broader" \
        and type(triple['s']['value']) == unicode \
        and triple['s']['value'].startswith("http://dbpedia.org/resource/Category:") \
        and triple['s']['value'].find('People') == -1:
            #print(triple['s']['value'])
            queue.append(triple['s']['value'])
            distinct_dbpedia_category.add(triple['s']['value'])
        elif triple['o']['value'] == resource \
        and triple['p']['value'] == u"http://purl.org/dc/terms/subject" \
        and type(triple['s']['value']) == unicode :
            distinct_dbpedia_places.add(triple['s']['value'])

for uri in distinct_dbpedia_category:
    print(uri)

http://dbpedia.org/resource/Category:Streets_in_Nice
http://dbpedia.org/resource/Category:Museums_in_Nice
http://dbpedia.org/resource/Category:Nice
http://dbpedia.org/resource/Category:Visitor_attractions_in_Nice
http://dbpedia.org/resource/Category:Education_in_Nice
http://dbpedia.org/resource/Category:Universities_in_Nice
http://dbpedia.org/resource/Category:OGC_Nice
http://dbpedia.org/resource/Category:OGC_Nice_managers
http://dbpedia.org/resource/Category:Buildings_and_structures_in_Nice
http://dbpedia.org/resource/Category:Open_de_Nice_Côte_d'Azur
http://dbpedia.org/resource/Category:Transport_in_Nice
http://dbpedia.org/resource/Category:Sportspeople_from_Nice
http://dbpedia.org/resource/Category:OGC_Nice_templates
http://dbpedia.org/resource/Category:Sport_in_Nice
http://dbpedia.org/resource/Category:OGC_Nice_matches
http://dbpedia.org/resource/Category:History_of_Nice
http://dbpedia.org/resource/Category:OGC_Nice_players
http://dbpedia.org/resource/Category:Media_in_Nice


In [43]:
final_poi = set()

for resource in distinct_dbpedia_places:
    sparql.setQuery("""
    select distinct ?uri where{

filter(?uri = <%s>).
{ ?uri a <http://dbpedia.org/ontology/Place> } 
UNION 
{ ?uri a <http://dbpedia.org/class/yago/YagoLegalActorGeo> }
UNION
{ ?uri a <http://dbpedia.org/class/yago/YagoPermanentlyLocatedEntity> }
minus 
{?uri a <http://dbpedia.org/ontology/Person>}
minus
{?uri a <http://dbpedia.org/class/yago/Person100007846>}
}
    """ % resource)
    results = sparql.query().convert()
    if len(results['results']['bindings']) > 0:
        final_poi.add(resource)

In [44]:
print(len(final_poi))

62


# ETL 

In [133]:
with open('source_data/dbpedia_poi_match.json', 'r') as pois:
    set_filename = Path("./tmp/raw_0.csv")
    raw_fp = open(str(set_filename), 'ab+')
    places = json.load(pois)
    
    distinct_dbpedia_uri = set([x[3] for x in places])
    print(len(distinct_dbpedia_uri))
    
    #insert all starting resources into the queue and the known set
    csv_out=csv.writer(raw_fp,)
    for row in distinct_dbpedia_uri:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8],)
    raw_fp.close()

47


In [151]:
from pathlib import Path
import os

In [152]:
def get_end(i):
    end_filename = Path("./tmp/end_" + str(i))
    return end_filename.exists()

In [153]:
def create_end(i):
    end_filename = Path("./tmp/end_" + str(i))
    open(str(end_filename), 'w',0).close()


In [154]:
def load_set_processed(i):
    set_filename = Path("./tmp/set_processed_" + str(i) + '.csv')
    if not set_filename.exists():
        open(str(set_filename), 'w',0).close()
    uris_fp = open(str(set_filename),'r',0)
    reader=csv.reader(uris_fp,)
    distinct_uris = set([unicode(x[0],'utf-8') for x in reader])
    uris_fp.close()
    uris_fp = open(str(set_filename),'wb',0)
    csv_out=csv.writer(uris_fp,)
    for row in distinct_uris:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8],)
    return distinct_uris, uris_fp

In [155]:
def load_raw_uri(i):
    set_filename = Path("./tmp/raw_" + str(i) + '.csv')
    if not set_filename.exists():
        open(str(set_filename), 'w',0).close()
    uris_fp = open(str(set_filename),'r',0)
    reader=csv.reader(uris_fp,)
    distinct_uris = set([unicode(x[0],'utf-8') for x in reader])
    uris_fp.close()
    
    uris_fp = open(str(set_filename),'wb',0)
    csv_out=csv.writer(uris_fp,)
    for row in distinct_uris:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8],)
    return distinct_uris, uris_fp

In [193]:
def process_uri(uri,set_processed,set_processed_fp,global_set, next_set_to_process,next_set_to_process_fp):
    banned_properties = [u'http://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'http://dbpedia.org/ontology/wikiPageRedirects', \
                     u'http://www.w3.org/2002/07/owl#sameAs', \
                     u'http://xmlns.com/foaf/spec/', \
                     u'https://dbpedia.org/ontology/wikiPageDisambiguates', \
                     u'https://dbpedia.org/ontology/wikiPageRedirects', \
                     u'https://www.w3.org/2002/07/owl#sameAs', \
                     u'https://xmlns.com/foaf/spec/']
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    sparql.setQuery("""describe <%s>""" % uri)
    results = sparql.query().convert()
    csv_out=csv.writer(next_set_to_process_fp,)
    for triple in results['results']['bindings']:
        if triple['p']['value'] not in banned_properties:
            if triple['o']['value'] == uri and type(triple['s']['value']) == unicode and triple['s']['value'].startswith("http://dbpedia.org/resource/"):
                if not triple['s']['value'] in next_set_to_process:
                    uri_utf8 = triple['s']['value'].encode('utf-8')
                    csv_out.writerow([uri_utf8],)
                    #print('add uri in next set to process %s' % triple['s']['value'])
                    next_set_to_process.add(triple['s']['value'])

            elif triple['s']['value'] == uri and type(triple['o']['value']) == unicode and triple['o']['value'].startswith("http://dbpedia.org/resource/"):
                #print(triple['o']['value'])
                if not triple['o']['value'] in next_set_to_process:
                    uri_utf8 = triple['o']['value'].encode('utf-8')
                    csv_out.writerow([uri_utf8],)
                    #print('add uri in next set to process %s' % triple['o']['value'])
                    next_set_to_process.add(triple['o']['value'])
    # add uri to set_processed and global_processed
    csv_out=csv.writer(set_processed_fp,)
    uri_utf8 = uri.encode('utf-8')
    csv_out.writerow([uri_utf8],)
    set_processed.add(uri)
    global_set.add(uri)


In [194]:
global_set = set()
for i in range(3):
    if get_end(i) == True:
        # I already processed all uri in set_i, which means that I can put them in the global set
        set_processed, set_processed_fp = load_set_processed(i)
        #print(set_processed_i)
        global_set |= set_processed
    else:
        # Last time I had some problems in processing the uri in step_i
        set_to_process,set_to_process_fp = load_raw_uri(i)
        set_to_process_fp.close()
        
        set_processed, set_processed_fp = load_set_processed(i)
        global_set |= set_processed
        set_not_processed_uri = set_to_process - global_set
        
        next_set_to_process,next_set_to_process_fp = load_raw_uri(i+1)
        
        for uri in set_not_processed_uri:
            process_uri(uri,set_processed,set_processed_fp,global_set, next_set_to_process,next_set_to_process_fp)
        set_processed_fp.close()
        next_set_to_process_fp.close()
        create_end(i)
        print(len(global_set))

47
646


KeyboardInterrupt: 

In [ ]:
import re
import Queue
import json
import collections
from geopy.distance import vincenty
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint
import json
from SPARQLWrapper import SPARQLWrapper,TURTLE,JSON
import time
import csv

with open('resources.csv', 'a+') as resources_fp:
    csv_out=csv.writer(resources_fp,)
    offset = 0
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    end = False
    while end is False:
        print(offset)
        sparql.setQuery("""
        select distinct ?s where {

        ?s ?o ?p .
        filter contains(str(?s),"http://dbpedia.org/resource")

        }
        limit 10000
        offset %d
            """ % offset)
        
        results = sparql.query().convert()
        if len(results['results']['bindings']) > 0:
            for result in results['results']['bindings']:
                uri_utf8 = result['s']['value'].encode('utf-8')
                csv_out.writerow([uri_utf8],)
            offset += 10000
        else:
            end = True

In [ ]:
import re
import Queue
import json
import collections
from geopy.distance import vincenty
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint
import json
from SPARQLWrapper import SPARQLWrapper,TURTLE,JSON
import time
import csv

with open('resources_category.csv', 'a+') as resources_fp:
    csv_out=csv.writer(resources_fp,)
    offset = 0
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    end = False
    while end is False:
        print(offset)
        sparql.setQuery("""
        SELECT distinct ?category WHERE { ?category a <http://www.w3.org/2004/02/skos/core#Concept> } limit 10000 offset %d
            """ % offset)
        
        results = sparql.query().convert()
        if len(results['results']['bindings']) > 0:
            for result in results['results']['bindings']:
                uri_utf8 = result['category']['value'].encode('utf-8')
                csv_out.writerow([uri_utf8],)
            offset += 10000
        else:
            end = True

In [ ]:
SELECT ?category WHERE { ?category a <http://www.w3.org/2004/02/skos/core#Concept> } 

In [50]:
import wget

In [ ]:
with open('sorted_resources_category.csv','r') as uris_fp:
    reader=csv.reader(uris_fp,)

    uris = [unicode(x[0],'utf-8') for x in reader]
    count = 1
    for uri in uris:
        uri_rdf = (uri.encode("UTF-8") + '.rdf').replace('http://dbpedia.org/resource/', 'http://dbpedia.org/data/',1)
        wget.download(uri_rdf,'out/')
        print(count)
        count += 1

In [ ]:
import os, os.path

# simple version for working with CWD
print len([name for name in os.listdir('.') if os.path.isfile(name)])
    

# gevent 

In [ ]:
#!/usr/bin/python
# Copyright (c) 2009 Denis Bilenko. See LICENSE for details.

"""Spawn multiple workers and wait for them to complete"""
from __future__ import print_function
import gevent
import wget
import csv
from gevent import monkey

# patches stdlib (including socket and ssl modules) to cooperate with other greenlets
monkey.patch_all()

import sys


def download(url):
    wget.download(url,'out/')

with open('sorted_resources_category.csv','r') as uris_fp:
    reader=csv.reader(uris_fp,)

    uris = [unicode(x[0],'utf-8') for x in reader]
    uris_rdf = [(uri.encode("UTF-8") + '.rdf').replace('http://dbpedia.org/resource/', 'http://dbpedia.org/data/',1) for uri in uris]
    
    jobs = [gevent.spawn(download, _url) for _url in uris_rdf]
    gevent.wait(jobs)

The history saving thread hit an unexpected error (LoopExit('This operation would block forever', <Hub at 0x12b3d0910 select pending=0 ref=0>)).History will not be written to the database.


Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/gevent/greenlet.py", line 536, in run
    result = self._run(*self.args, **self.kwargs)
  File "<ipython-input-55-d0bd1ed94208>", line 16, in download
    wget.download(url,'out/')
  File "/usr/local/lib/python2.7/site-packages/wget.py", line 526, in download
    (tmpfile, headers) = ulib.urlretrieve(binurl, tmpfile, callback)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/urllib.py", line 98, in urlretrieve
    return opener.retrieve(url, filename, reporthook, data)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/urllib.py", line 245, in retrieve
    fp = self.open(url, data)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/urllib.py", line 213, in open
    return getattr(self, name)(url)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2

In [7]:
from urllib2 import urlopen, URLError, HTTPError
import os

In [13]:
def dlfile(url):
    # Open the url
    try:
        f = urlopen(url)
        print("downloading " + url)
        # Open our local file for writing
        with open('out/' + os.path.basename(url), "wb") as local_file:
            local_file.write(f.read())

    #handle errors
    except HTTPError, e:
        print("HTTP Error:", e.code, url)
    except URLError, e:
        print("URL Error:", e.reason, url)

In [14]:
import wget
import csv
with open('sorted_resources_category.csv','r') as uris_fp:
    reader=csv.reader(uris_fp,)

    uris = [unicode(x[0],'utf-8') for x in reader]
    uris_rdf = [(uri.encode("UTF-8") + '.rdf').replace('http://dbpedia.org/resource/', 'http://dbpedia.org/data/',1) for uri in uris]
    count =0
    for uri in uris_rdf[:1]:
        dlfile(uri)
        print count
        count +=1

downloading http://dbpedia.org/data/Category:%22The_Raven%22,_Édouard_Manet's_illustrations_(featured_picture_set).rdf
Category:%22The_Raven%22,_Édouard_Manet's_illustrations_(featured_picture_set).rdf
0


In [ ]:
from gevent.pool import Pool
import logging
rows = [ ... a large list of stuff ...]
CONCURRENCY = 200 # run 200 greenlets at once or whatever you want
pool = Pool(CONCURRENCY)
count = 0

def do_work_function(param1,param2):
   print param1 + param2

for row in rows:
  count += 1 # for logging purposes to track progress
  logging.info(count)
  pool.spawn(do_work_function,param1,param2) # blocks here when pool size == CONCURRENCY

pool.join() #blocks here until the last 200 are complete

# Expand_resources

In [1]:
import re
import Queue
import json
import collections
from geopy.distance import vincenty
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pprint import pprint
import json
from SPARQLWrapper import SPARQLWrapper,TURTLE,JSON
import time
import csv

In [6]:
# this set contains all resources downloaded
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

set_uri_known = set()
queue_uri_to_expand = collections.deque()

with open('resources_to_expand.csv','r') as input_fp:
                reader=csv.reader(input_fp,)
                set_uri_known = set([unicode(x[0],'utf-8') for x in reader])
count = len(set_uri_known)
for uri in set_uri_known:
    queue_uri_to_expand.append(uri)  
while len(queue_uri_to_expand) > 0:
    resource = queue_uri_to_expand.popleft()
    sparql.setQuery("""select distinct ?o where {<%s> <http://purl.org/dc/terms/subject> ?o}""" % resource)
    results = sparql.query().convert()
    for triple in results['results']['bindings']:
        if triple['o']['value'] not in set_uri_known:
            set_uri_known.add(triple['o']['value'])
            queue_uri_to_expand.append(triple['o']['value'])
            count +=1
            if count%1000 ==0:
                print(count)
                #print(triple['o']['value'])

15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000


KeyboardInterrupt: 

In [ ]:
import json
import csv
with open('source_data/dbpedia_poi_match.json', 'r') as pois:
    with open('source_data/1_6_doremus_dbpedia_works.csv', 'r') as works:
        with open('source_data/1_7_doremus_dbpedia_artists.csv', 'r') as artists:
            with open('resources_to_expand.csv','w') as out_fp:
                places = json.load(pois)
                distinct_dbpedia_uri = set([x[3] for x in places])

                reader=csv.reader(artists,)
                next(reader, None)  # skip the headers
    
                distinct_dbpedia_uri |= set([unicode(x[1],'utf-8') for x in reader])
                reader=csv.reader(works,)
                next(reader, None)  # skip the headers

                distinct_dbpedia_uri |= set([unicode(x[1],'utf-8') for x in reader])

                csv_out=csv.writer(out_fp,)
                for row in distinct_dbpedia_uri:
                    row_utf8 = row.encode('utf-8')
                    csv_out.writerow([row_utf8])


In [16]:
a = ("""%s %s""" % (resource,resource))
a

u'http://dbpedia.org/resource/Category:Catalonia_international_footballers http://dbpedia.org/resource/Category:Catalonia_international_footballers'

In [18]:
with open('resources_expanded.csv','wb') as out_fp:
    csv_out=csv.writer(out_fp,)
    for row in set_uri_known:
        row_utf8 = row.encode('utf-8')
        csv_out.writerow([row_utf8])
